In [1]:
import locale
print(locale.getpreferredencoding())
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

UTF-8


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!ls "/content/drive/My Drive/nlp"

cleaned-kaggle2.csv		glove_twitter
cleaned-proppy2.csv		models
cleaned-wellfake2.csv		prepped-data
data				wellfake_lr_title_vectorizer.pickle
fake_news_classification.ipynb


In [4]:
import os
current_dir = "/content/drive/MyDrive/nlp"
os.chdir("/content/drive/MyDrive/nlp") 

In [5]:
!pwd

/content/drive/MyDrive/nlp


In [6]:
!pip install -q keras

In [7]:
# setup
import sys
import subprocess
import re
import os
import spacy
import numpy as np
import pandas as pd
import pickle


from bs4 import BeautifulSoup
from spacy.lang.en import English

import en_core_web_sm
nlp = en_core_web_sm.load()

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))


import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


#Importuojame paketus
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

!pip install --upgrade protobuf

import keras
import tensorflow as tf
import tensorflow_hub as hub
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [8]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.12.0
Eager mode:  True
Hub version:  0.13.0
GPU is available


## Importuojame išvalytus duomenis

In [9]:
kaggle_data = pd.read_csv('prepped-data/cleaned-kaggle2.csv')
kaggle_data.dropna(inplace = True)
kaggle_data.head()


,label,title,text,cleaned_text,cleaned_title
0,1,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,donald trump wish american happy new year leav...,donald trump sends embarrassing new year eve m...
1,1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,house intelligence committee chairman devin nu...,drunk bragging trump staffer started russian c...
2,1,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",friday revealed former milwaukee sheriff david...,sheriff david clarke becomes internet joke thr...
3,1,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",christmas day donald trump announced would bac...,trump obsessed even obamas name coded website ...
4,1,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,pope francis used annual christmas day message...,pope francis called donald trump christmas speech


In [10]:
wellfake_data = pd.read_csv('prepped-data/cleaned-wellfake2.csv')
wellfake_data.dropna(inplace = True)
wellfake_data.head()

,title,text,label,cleaned_text,cleaned_title
0,UNBELIEVABLE! OBAMAS ATTORNEY GENERAL SAYS MOS...,"Now, most of the demonstrators gathered last ...",1,demonstrator gathered last night exercising co...,unbelievable obamas attorney general say charl...
1,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,dozen politically active pastor came private d...,bobby jindal raised hindu us story christian c...
2,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,r sarmat missile dubbed satan replace s fly mi...,satan russia unvelis image terrifying new supe...
3,DR BEN CARSON TARGETED BY THE IRS: I never had...,DR. BEN CARSON TELLS THE STORY OF WHAT HAPPENE...,1,dr ben carson tell story happened spoke obama,dr ben carson targeted irs never audit spoke n...
5,Sports Bar Owner Bans NFL GamesWill Show Only ...,"The owner of the Ringling Bar, located south o...",1,owner ringling bar located south white sulphur...,sport bar owner ban nfl gameswill show true am...


In [11]:
proppy_data = pd.read_csv('prepped-data/cleaned-proppy2.csv')
proppy_data.dropna(inplace = True)
proppy_data.head()

,text,title,label,cleaned_text,cleaned_title
0,Convened to examine the causes of civil unrest...,Report: No progress for African-Americans on h...,0.0,convened examine cause civil unrest black comm...,report progress africanamericans homeownership...
1,Discriminating against someone on the basis of...,Employers can't discriminate based on sexual o...,0.0,discriminating someone basis sexual orientatio...,employer cant discriminate based sexual orient...
2,"Bill Cosby's 44-year-old daughter, Ensa Cosby,...","Bill Cosby's daughter Ensa, 44, dies in Massac...",0.0,bill cosbys yearold daughter ensa cosby died m...,bill cosbys daughter ensa dy massachusetts
3,"The fast-moving, powerful theatrical locomotiv...","'Murder On The Orient Express' A Funny, Fast-P...",0.0,fastmoving powerful theatrical locomotive murd...,murder orient express funny fastpaced thrill r...
4,It's Friday. It's National Pizza Day. Grab lif...,You Can Celebrate National Pizza Day Right Here,0.0,friday national pizza day grab life slice conn...,celebrate national pizza day right


In [51]:
import matplotlib.pyplot as plt
import seaborn as sns

def word_count_plot(df, col='cleaned_text'):
    # Set the style to grayscale
    sns.set(style='white', palette='gray')

    # Create the subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Plot histogram for fake news
    text_len_fake = df[df['label'] == 0][col].str.split().map(lambda x: len(x))
    ax1.hist(text_len_fake, bins=30, color='lightgray', edgecolor='black')
    ax1.set_title('Fake News')
    ax1.set_xlabel('Word Count')
    ax1.set_ylabel('Frequency')
    ax1.set_xticks(range(0, max(text_len_fake) + 1, 250))  # Set x-axis ticks with spacing of 250
    ax1.tick_params(axis='x', rotation=90)  # Rotate x-axis tick labels by 90 degrees

   

    # Plot histogram for real news
    text_len_real = df[df['label'] == 1][col].str.split().map(lambda x: len(x))
    ax2.hist(text_len_real, bins=30, color='lightgray', edgecolor='black')
    ax2.set_title('Real News')
    ax2.set_xlabel('Word Count')
    ax2.set_ylabel('Frequency')
    ax2.set_xticks(range(0, max(text_len_real) + 1, 250))  # Set x-axis ticks with spacing of 250
    ax2.tick_params(axis='x', rotation=90)  # Rotate x-axis tick labels by 90 degrees

       # Calculate and log average word count
    avg_fake = sum(text_len_fake) / len(text_len_fake)
    avg_real = sum(text_len_real) / len(text_len_real)
    plt.text(0.5, -0.15, f'Average Word Count (Fake News): {avg_fake:.2f}\nAverage Word Count (Real News): {avg_real:.2f}',
             horizontalalignment='center', verticalalignment='center', transform=ax2.transAxes)
    

    # Set the title
    fig.suptitle(f'Words in {col}', fontsize=16, fontweight='bold')

    # Remove spines
    sns.despine()

    # Display the plot
    plt.tight_layout()
    plt.show()

## Get word length

In [75]:
def word_length_plot(df, col='cleaned_text'):
    # Set the style to grayscale
    sns.set(style='white', palette='gray')

    # Create the subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Plot histogram for original text
    word_original = df[df['label'] == 1][col].str.split().apply(lambda x: [len(i) for i in x if len(i) <= 7])
    sns.distplot(word_original.map(lambda x: np.mean(x)), ax=ax1, color='gray')
    ax1.set_title('Original Text')
    ax1.set_xlabel('Average Word Length')
    ax1.set_ylabel('Density')

    # Plot histogram for fake text
    word_fake = df[df['label'] == 0][col].str.split().apply(lambda x: [len(i) for i in x if len(i) <= 7])
    sns.distplot(word_fake.map(lambda x: np.mean(x)), ax=ax2, color='gray')
    ax2.set_title('Fake Text')
    ax2.set_xlabel('Average Word Length')
    ax2.set_ylabel('Density')

    # Set the title
    fig.suptitle('Average Word Length in Each Text', fontsize=16, fontweight='bold')

    # Remove spines
    sns.despine()

    # Display the plot
    plt.tight_layout()
    plt.show()

### Created a combined attribute from main text and title

In [12]:
kaggle_data['cleaned_combined'] = kaggle_data['title'] + ' ' + kaggle_data['text']
wellfake_data['cleaned_combined'] = wellfake_data['title'] + ' ' + wellfake_data['text']
# proppy_data['cleaned_combined'] = proppy_data['title'] + ' ' + proppy_data['text']


#### Train-test-split, vectorization

In [15]:
from sklearn.model_selection import train_test_split
import pickle

# Can also be 'cleaned_title', 'cleaned_combined' or 'cleaned_text'
# returns: X_train, X_test, y_train, y_test
def create_train_test_split(dataset, attribute):
  x = getattr(dataset, attribute)
  y = dataset['label']
  return train_test_split(x, y, test_size=0.20, shuffle=True, random_state=0)  

def vectorize(vectorizer, x_train, x_test,filename):
  x_train_vect = vectorizer.fit_transform(x_train)
  x_test_vect = vectorizer.transform(x_test)
  
  # save vectorizer
  vec_filename = f'{filename}_vectorizer.pickle'
  pickle.dump(vectorizer, open(f'models/{vec_filename}', 'wb'))

  return x_train_vect, x_test_vect

#### Classical ML utils

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from joblib import dump


# Classical ML 
def train_classical_model(model, x_train, y_train, x_test, y_test):
  model.fit(x_train, y_train)
  y_pred = model.predict(x_test)
  acc = accuracy_score(y_test, y_pred)
  # Evaluate model
  print("Accuracy:", acc)
  target_names = ['True', 'Fake']
  print(classification_report(y_test, y_pred, target_names=target_names, digits=3))

  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
  specificity = tn / (tn+fp)
  print(f'Specificity {specificity}')
  # Save model 

def save_classical_model(model, filename):
  dump(model, f'models/{filename}.joblib') 
  print(f'Model {filename} saved!')


# Combining function
def train_and_eval_model(dataset, attribute, vectorizer, model, filename='classical_model'):
  x_train, x_test, y_train, y_test = create_train_test_split(dataset, attribute)
  x_train_vect, x_test_vect = vectorize(vectorizer, x_train, x_test, filename)
  train_classical_model(model, x_train_vect, y_train, x_test_vect, y_test)
  save_classical_model(model, filename)

#### Deep learning utils

In [ ]:
GLOVE_EMBEDDING = "./glove_twitter/glove.twitter.27B.100d.txt"

In [ ]:
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer

def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype = "float32")


def tokenize(x_train, x_test):
  tokenizer = Tokenizer(num_words = 10000)
  tokenizer.fit_on_texts(x_train)
  tokenized_train = tokenizer.texts_to_sequences(x_train)
  tokenized_test = tokenizer.texts_to_sequences(x_test)

  x_train = pad_sequences(tokenized_train, maxlen = 300)
  x_test = pad_sequences(tokenized_test, maxlen = 300)
  return x_train, x_test, tokenizer


def generate_embedding_matrix(tokenizer):

  embeddings_index = dict(get_coefs(*g.rstrip().rsplit(" ")) for g in open(GLOVE_EMBEDDING, errors="ignore"))
  embeddings = np.stack(embeddings_index.values())
  embedding_mean, embedding_std = embeddings.mean(), embeddings.std()
  embedding_size = embeddings.shape[1]

  word_index = tokenizer.word_index
  nb_words = min(10000, len(word_index))

  embedding_matrix = np.random.normal(embedding_mean, embedding_std, (nb_words, embedding_size))
  for word, i in word_index.items():
      if i >= 10000:
          continue
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
  
  return embedding_matrix


In [ ]:
from keras.layers import Bidirectional

# LSTM
def construct_lstm(embedding_matrix):
  model = Sequential()
  model.add(Embedding(10000,
                      output_dim = 100,
                      weights = [embedding_matrix],
                      input_length = 300,
                      trainable = False))

  model.add(LSTM(units = 128,
                return_sequences = True,
                recurrent_dropout = 0,
                dropout = 0.3,
                activation = "tanh",
                recurrent_activation = "sigmoid",
                  unroll=False,
                  use_bias = True
                  ))

  model.add(LSTM(units = 64,
                recurrent_dropout = 0,
                dropout = 0.15,
                activation = "tanh",
                recurrent_activation = "sigmoid",
                unroll=False,
                use_bias = True))

  model.add(Dense(units = 32,
                  activation = "relu"))

  model.add(Dense(1,
                  activation = "sigmoid"))

  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
                loss = "binary_crossentropy",
                metrics = ["accuracy"])

  return model

# BiLSTM
def construct_bi_lstm(embedding_matrix):
  model = Sequential()
  model.add(Embedding(10000,
                      output_dim = 100,
                      weights = [embedding_matrix],
                      input_length = 300,
                      trainable = False))
   
  model.add(Bidirectional(LSTM(64, return_sequences = True,  recurrent_dropout=0, dropout = 0.15,
                activation = "tanh",
                recurrent_activation = "sigmoid",
                unroll=False,
                use_bias = True)))
  model.add(Bidirectional(LSTM(32,  recurrent_dropout=0)))
  model.add(Dense(units = 64, activation = "relu"))
  model.add(Dense(units = 1, activation = "sigmoid"))
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss = "binary_crossentropy",
              metrics = ["accuracy"])
  
  return model

In [ ]:
from keras.models import Model


def get_selected_model(embedding_matrix, model_type='placeholder'):
  if model_type=='bi':
    return construct_bi_lstm(embedding_matrix)
  else:
    return construct_lstm(embedding_matrix)

def train_dl_model(model, x_train, y_train, x_test, y_test):
  lr_reduce = ReduceLROnPlateau(monitor = "val_loss", patience = 5, factor = 0.5, min_lr = 0.00001)

  history = model.fit(x_train,
                      y_train,
                      batch_size = 128,
                      validation_data = (x_test, y_test),
                      epochs = 10,
                      callbacks = [lr_reduce])
   

def evaluate_dl_model(model, x_train, y_train, x_test, y_test):
   # Evaluation
  print("Model accuracy on training data - " , model.evaluate(x_train, y_train)[1]*100 , "%")
  print("Model accuracy on testing data - " , model.evaluate(x_test, y_test)[1]*100 , "%")

  prediction = (model.predict(x_test) > 0.5).astype("int32")
  print(classification_report(y_test, prediction, target_names = ['Fake','Real'], digits=3))
  tn, fp, fn, tp = confusion_matrix(y_test, prediction).ravel()
  specificity = tn / (tn+fp)
  print(f'Specificity {specificity}')

def save_dl_model(model, filename):
  model.save(f'models/{filename}.h5')
  print(f'Model {filename} saved!')



# types: 'bi' - bidirectional, 
## can currently be a flag, but more types will be added in the future
def train_and_evalute_dl_model(dataset, attribute, model_type='placeholder', filename='dl_model' ):
  x_train, x_test, y_train, y_test = create_train_test_split(dataset, attribute)
  x_train, x_test, tokenizer = tokenize(x_train, x_test)
  embedding_matrix = generate_embedding_matrix(tokenizer)
  model = get_selected_model(embedding_matrix, model_type)

  model.summary()
  train_dl_model(model, x_train, y_train, x_test, y_test)
  evaluate_dl_model(model, x_train, y_train, x_test, y_test)
  save_dl_model(model, filename)


## Kaggle

#### TFIDF + NB & Text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
nb = MultinomialNB()

train_and_eval_model(kaggle_data, 'cleaned_text', vectorizer, nb)

Accuracy: 0.9273410180028494
              precision    recall  f1-score   support

        True      0.895     0.983     0.937      4247
        Fake      0.977     0.859     0.914      3474

    accuracy                          0.927      7721
   macro avg      0.936     0.921     0.926      7721
weighted avg      0.932     0.927     0.927      7721

Specificity 0.9832823169295973


#### TFIDF + NB & Title

In [ ]:
train_and_eval_model(kaggle_data, 'cleaned_title', vectorizer, nb)

Accuracy: 0.932780727884989
              precision    recall  f1-score   support

        True      0.933     0.945     0.939      4247
        Fake      0.932     0.918     0.925      3474

    accuracy                          0.933      7721
   macro avg      0.933     0.931     0.932      7721
weighted avg      0.933     0.933     0.933      7721

Specificity 0.9451377442900871


#### TFIDF + NB & Combined



In [ ]:
train_and_eval_model(kaggle_data, 'cleaned_combined', vectorizer, nb)

Accuracy: 0.9396451236886414
              precision    recall  f1-score   support

        True      0.927     0.967     0.946      4247
        Fake      0.957     0.906     0.931      3474

    accuracy                          0.940      7721
   macro avg      0.942     0.937     0.939      7721
weighted avg      0.940     0.940     0.939      7721

Specificity 0.9668000941841299


### **Logistic** Regression

#### TFIDF + LR & Text



In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train_and_eval_model(kaggle_data, 'cleaned_text', vectorizer, lr)

Accuracy: 0.9847170055692268
              precision    recall  f1-score   support

        True      0.981     0.991     0.986      4247
        Fake      0.989     0.977     0.983      3474

    accuracy                          0.985      7721
   macro avg      0.985     0.984     0.985      7721
weighted avg      0.985     0.985     0.985      7721

Specificity 0.9910525076524606


#### TFIDF + LR & Title

In [ ]:
train_and_eval_model(kaggle_data, 'cleaned_title', vectorizer, lr)

Accuracy: 0.9325216940810775
              precision    recall  f1-score   support

        True      0.927     0.953     0.940      4247
        Fake      0.940     0.908     0.924      3474

    accuracy                          0.933      7721
   macro avg      0.933     0.930     0.932      7721
weighted avg      0.933     0.933     0.932      7721

Specificity 0.9526724746880151


#### TFIDF + LR & Combined

In [ ]:
train_and_eval_model(kaggle_data, 'cleaned_combined', vectorizer, lr)

Accuracy: 0.984198937961404
              precision    recall  f1-score   support

        True      0.980     0.992     0.986      4247
        Fake      0.990     0.975     0.982      3474

    accuracy                          0.984      7721
   macro avg      0.985     0.983     0.984      7721
weighted avg      0.984     0.984     0.984      7721

Specificity 0.9919943489522015




---



#### Glove + LSTM & Text

In [ ]:
seq = Sequential()
train_and_evalute_dl_model(kaggle_data, 'cleaned_text', seq, filename="kaggle_lstm_text")

<ipython-input-19-1bbc3a5794c0>:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  train_and_evalute_dl_model(kaggle_data, 'cleaned_text', seq, filename="kaggle_lstm_text")


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 300, 128)          117248    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,168,769
Trainable params: 168,769
Non-trainable params: 1,000,000
_________________________________________________________________
Epoch 1/10
242/242 [==================

#### Glove + LSTM & Title

In [ ]:
seq = Sequential()
train_and_evalute_dl_model(kaggle_data, 'cleaned_title', seq)

<ipython-input-37-080d608a1352>:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  train_and_evalute_dl_model(kaggle_data, 'cleaned_title', seq)


Epoch 1/10
242/242 [==============================] - 13s 39ms/step - loss: 0.3270 - accuracy: 0.8558 - val_loss: 0.2502 - val_accuracy: 0.8966 - lr: 0.0010
Epoch 2/10
242/242 [==============================] - 9s 36ms/step - loss: 0.2329 - accuracy: 0.9030 - val_loss: 0.2030 - val_accuracy: 0.9170 - lr: 0.0010
Epoch 3/10
242/242 [==============================] - 9s 37ms/step - loss: 0.2031 - accuracy: 0.9174 - val_loss: 0.1803 - val_accuracy: 0.9270 - lr: 0.0010
Epoch 4/10
242/242 [==============================] - 9s 36ms/step - loss: 0.1829 - accuracy: 0.9242 - val_loss: 0.1677 - val_accuracy: 0.9317 - lr: 0.0010
Epoch 5/10
242/242 [==============================] - 9s 36ms/step - loss: 0.1704 - accuracy: 0.9300 - val_loss: 0.1767 - val_accuracy: 0.9276 - lr: 0.0010
Epoch 6/10
242/242 [==============================] - 9s 36ms/step - loss: 0.1581 - accuracy: 0.9366 - val_loss: 0.1549 - val_accuracy: 0.9400 - lr: 0.0010
Epoch 7/10
242/242 [==============================] - 9s 36ms/s

#### Glove + LSTM & Combined

In [ ]:
seq = Sequential()
train_and_evalute_dl_model(kaggle_data, 'cleaned_combined', seq)

<ipython-input-39-d727d62dfa25>:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  train_and_evalute_dl_model(kaggle_data, 'cleaned_combined', seq)


Epoch 1/10
242/242 [==============================] - 13s 39ms/step - loss: 0.2589 - accuracy: 0.8973 - val_loss: 0.1906 - val_accuracy: 0.9282 - lr: 0.0010
Epoch 2/10
242/242 [==============================] - 9s 36ms/step - loss: 0.1843 - accuracy: 0.9316 - val_loss: 0.1847 - val_accuracy: 0.9346 - lr: 0.0010
Epoch 3/10
242/242 [==============================] - 9s 37ms/step - loss: 0.1733 - accuracy: 0.9324 - val_loss: 0.2055 - val_accuracy: 0.9272 - lr: 0.0010
Epoch 4/10
242/242 [==============================] - 9s 37ms/step - loss: 0.1367 - accuracy: 0.9500 - val_loss: 0.1002 - val_accuracy: 0.9678 - lr: 0.0010
Epoch 5/10
242/242 [==============================] - 9s 36ms/step - loss: 0.1016 - accuracy: 0.9662 - val_loss: 0.1114 - val_accuracy: 0.9637 - lr: 0.0010
Epoch 6/10
242/242 [==============================] - 9s 36ms/step - loss: 0.1012 - accuracy: 0.9659 - val_loss: 0.0619 - val_accuracy: 0.9784 - lr: 0.0010
Epoch 7/10
242/242 [==============================] - 9s 36ms/s

#### Glove + BiLSTM & Text

In [ ]:
train_and_evalute_dl_model(kaggle_data, 'cleaned_text', model_type='bi')

NameError: ignored

#### Glove + BiLSTM & Title

In [ ]:
seq = Sequential()
train_and_evalute_dl_model(kaggle_data, 'cleaned_title', model_type='bi')

<ipython-input-102-9b78ff0ef1dd>:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  train_and_evalute_dl_model(kaggle_data, 'cleaned_title', model_type='bi')


Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_28 (Embedding)    (None, 300, 100)          1000000   
                                                                 
 bidirectional_26 (Bidirecti  (None, 300, 128)         84480     
 onal)                                                           
                                                                 
 bidirectional_27 (Bidirecti  (None, 64)               41216     
 onal)                                                           
                                                                 
 dense_52 (Dense)            (None, 64)                4160      
                                                                 
 dense_53 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,129,921
Trainable params: 129,921
Non

#### Glove + BiLSTM & Combined

In [ ]:
seq = Sequential()
train_and_evalute_dl_model(kaggle_data, 'cleaned_combined', seq, model_type='bi')

<ipython-input-54-64b5cf3f8fff>:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  train_and_evalute_dl_model(kaggle_data, 'cleaned_combined', seq, model_type='bi')


Epoch 1/10
242/242 [==============================] - 18s 51ms/step - loss: 0.1877 - accuracy: 0.9292 - val_loss: 0.1069 - val_accuracy: 0.9631 - lr: 0.0010
Epoch 2/10
242/242 [==============================] - 11s 47ms/step - loss: 0.1227 - accuracy: 0.9564 - val_loss: 0.0805 - val_accuracy: 0.9728 - lr: 0.0010
Epoch 3/10
242/242 [==============================] - 11s 46ms/step - loss: 0.0848 - accuracy: 0.9713 - val_loss: 0.1477 - val_accuracy: 0.9509 - lr: 0.0010
Epoch 4/10
242/242 [==============================] - 11s 46ms/step - loss: 0.0700 - accuracy: 0.9775 - val_loss: 0.0542 - val_accuracy: 0.9815 - lr: 0.0010
Epoch 5/10
242/242 [==============================] - 11s 45ms/step - loss: 0.0520 - accuracy: 0.9825 - val_loss: 0.0437 - val_accuracy: 0.9839 - lr: 0.0010
Epoch 6/10
242/242 [==============================] - 11s 45ms/step - loss: 0.0404 - accuracy: 0.9865 - val_loss: 0.0441 - val_accuracy: 0.9837 - lr: 0.0010
Epoch 7/10
242/242 [==============================] - 11s 

# Wellfake **

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

lr = LogisticRegression()
train_and_eval_model(wellfake_data, 'cleaned_text', vectorizer, lr)

Accuracy: 0.9380294248773964
              precision    recall  f1-score   support

        True      0.946     0.940     0.943      4868
        Fake      0.929     0.936     0.933      4104

    accuracy                          0.938      8972
   macro avg      0.937     0.938     0.938      8972
weighted avg      0.938     0.938     0.938      8972

Specificity 0.9398110106820049
Model classical_model saved!


In [ ]:
train_and_eval_model(wellfake_data, 'cleaned_title', vectorizer, lr, filename="wellfake_lr_title")

Accuracy: 0.8808515381185912
              precision    recall  f1-score   support

        True      0.896     0.882     0.889      4868
        Fake      0.863     0.879     0.871      4104

    accuracy                          0.881      8972
   macro avg      0.880     0.881     0.880      8972
weighted avg      0.881     0.881     0.881      8972

Specificity 0.8824979457682827
Model wellfake_lr_title saved!


In [ ]:
train_and_eval_model(wellfake_data, 'cleaned_combined', vectorizer, lr, filename="wellfake_lr_combined")

Accuracy: 0.9478377173428444
              precision    recall  f1-score   support

        True      0.951     0.953     0.952      4868
        Fake      0.944     0.942     0.943      4104

    accuracy                          0.948      8972
   macro avg      0.948     0.947     0.947      8972
weighted avg      0.948     0.948     0.948      8972

Specificity 0.9529580936729664


### Naive Bayes

In [ ]:
nb = MultinomialNB()

train_and_eval_model(wellfake_data, 'cleaned_text', vectorizer, nb)

Accuracy: 0.765492643780651
              precision    recall  f1-score   support

        True      0.700     0.992     0.821      4868
        Fake      0.981     0.497     0.660      4104

    accuracy                          0.765      8972
   macro avg      0.841     0.744     0.740      8972
weighted avg      0.829     0.765     0.747      8972

Specificity 0.991988496302383


In [ ]:
train_and_eval_model(wellfake_data, 'cleaned_title', vectorizer, nb)

Accuracy: 0.8533214444939813
              precision    recall  f1-score   support

        True      0.827     0.922     0.872      4868
        Fake      0.893     0.772     0.828      4104

    accuracy                          0.853      8972
   macro avg      0.860     0.847     0.850      8972
weighted avg      0.857     0.853     0.852      8972

Specificity 0.9221446179129006


In [ ]:
train_and_eval_model(wellfake_data, 'cleaned_combined', vectorizer, nb)

Accuracy: 0.8174320106999554
              precision    recall  f1-score   support

        True      0.755     0.981     0.854      4868
        Fake      0.965     0.623     0.757      4104

    accuracy                          0.817      8972
   macro avg      0.860     0.802     0.806      8972
weighted avg      0.851     0.817     0.810      8972

Specificity 0.981101068200493


#### Glove & LSTM

##### LSTM

In [ ]:
train_and_evalute_dl_model(wellfake_data, 'cleaned_text')

<ipython-input-84-70d6bd1b63be>:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  train_and_evalute_dl_model(wellfake_data, 'cleaned_text')


Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 300, 100)          1000000   
                                                                 
 lstm_34 (LSTM)              (None, 300, 128)          117248    
                                                                 
 lstm_35 (LSTM)              (None, 64)                49408     
                                                                 
 dense_34 (Dense)            (None, 32)                2080      
                                                                 
 dense_35 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,168,769
Trainable params: 168,769
Non-trainable params: 1,000,000
_________________________________________________________________
Epoch 1/10
281/281 [=================

In [ ]:
train_and_evalute_dl_model(wellfake_data, 'cleaned_title')

<ipython-input-87-699a45f5eee1>:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  train_and_evalute_dl_model(wellfake_data, 'cleaned_title')


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 300, 100)          1000000   
                                                                 
 lstm_38 (LSTM)              (None, 300, 128)          117248    
                                                                 
 lstm_39 (LSTM)              (None, 64)                49408     
                                                                 
 dense_38 (Dense)            (None, 32)                2080      
                                                                 
 dense_39 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,168,769
Trainable params: 168,769
Non-trainable params: 1,000,000
_________________________________________________________________
Epoch 1/10
281/281 [=================

In [ ]:
train_and_evalute_dl_model(wellfake_data, 'cleaned_combined', 'wellfake_lstm_combined')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 lstm_4 (LSTM)               (None, 300, 128)          117248    
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,168,769
Trainable params: 168,769
Non-trainable params: 1,000,000
_________________________________________________________________
Epoch 1/10
281/281 [==================

In [ ]:
train_and_evalute_dl_model(wellfake_data, 'cleaned_text', model_type='bi')

<ipython-input-97-e65ffe719b60>:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  train_and_evalute_dl_model(wellfake_data, 'cleaned_text', model_type='bi')


Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, 300, 100)          1000000   
                                                                 
 bidirectional_20 (Bidirecti  (None, 300, 128)         84480     
 onal)                                                           
                                                                 
 bidirectional_21 (Bidirecti  (None, 64)               41216     
 onal)                                                           
                                                                 
 dense_46 (Dense)            (None, 64)                4160      
                                                                 
 dense_47 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,129,921
Trainable params: 129,921
Non

In [ ]:
train_and_evalute_dl_model(wellfake_data, 'cleaned_title', model_type='bi')

<ipython-input-98-308d6b9be4a7>:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  train_and_evalute_dl_model(wellfake_data, 'cleaned_title', model_type='bi')


Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_26 (Embedding)    (None, 300, 100)          1000000   
                                                                 
 bidirectional_22 (Bidirecti  (None, 300, 128)         84480     
 onal)                                                           
                                                                 
 bidirectional_23 (Bidirecti  (None, 64)               41216     
 onal)                                                           
                                                                 
 dense_48 (Dense)            (None, 64)                4160      
                                                                 
 dense_49 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,129,921
Trainable params: 129,921
Non

In [ ]:
train_and_evalute_dl_model(wellfake_data, 'cleaned_combined', model_type='bi', filename='wellfake_bilstm_combined')

<ipython-input-22-687410555f58>:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  train_and_evalute_dl_model(wellfake_data, 'cleaned_combined', model_type='bi', filename='wellfake_bilstm_combined')


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 bidirectional (Bidirectiona  (None, 300, 128)         84480     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,129,921
Trainable params: 129,921
Non-